In [2]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

#from google.colab import drive
#drive.mount('/content/gdrive')


def replaceNaNwithAvg(dataFrame, column_name):
	age_array = dataFrame[dataFrame[column_name]!=np.nan][column_name]
	dataFrame[column_name] = dataFrame[column_name].replace(np.nan,age_array.mean())
	return dataFrame

# file_path_train =  'titanic/train.csv'
# file_path_test = 'titanic/test.csv'

file_path_train = 'https://raw.githubusercontent.com/zhangleiu/Kaggle/master/train.csv'

titanic_df = pd.read_csv(file_path_train)
# print(titanic_df.head(10))
dataFrame = pd.read_csv(file_path_train, usecols=['Survived', 'Pclass', 'Sex', 'Age', \
 	'SibSp', 'Parch', 'Fare', 'Embarked'])
dataFrame.head(20)
# print (dataFrame.describe())
# print (dataFrame.isna().sum())
dataFrame = dataFrame.replace({'female': 0, 'male': 1})
dataFrame = dataFrame.replace({'Embarked': {'C': 0, 'Q': 1, 'S':2}})
dataFrame = replaceNaNwithAvg(dataFrame, "Age")
dataFrame['Embarked'] = dataFrame['Embarked'].replace(np.nan,3)
# print(dataFrame.isna().sum())
# print (dataFrame.corr())

dataArray = dataFrame.values
idx_output_columns=[0]
idx_feature_columns = [i for i in range(np.shape(dataArray)[1]) if i not in idx_output_columns]
feature_columns = dataArray[:,idx_feature_columns]
output_columns =  dataArray[:,0]
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()

max_tree = 100
accuracy = {'Bagging': [0]*max_tree, 'Boosting': [0]*max_tree, 'RandomForest': [0]*max_tree}
# num_trees = 100
for num_trees in range(max_tree): 
  #Bagging test
  model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees+1, random_state=seed)
  results = model_selection.cross_val_score(model, feature_columns, output_columns, cv=kfold)
  accuracy['Bagging'][num_trees] = results.mean()
  #Boosting test
  model = AdaBoostClassifier(base_estimator=cart, n_estimators=num_trees+1, random_state=seed)
  results = model_selection.cross_val_score(model, feature_columns, output_columns, cv=kfold)
  accuracy['Boosting'][num_trees] = results.mean()
  #Random Forest
  model = RandomForestClassifier(n_estimators = num_trees+1, max_depth = 2, random_state = 0)
  results = model_selection.cross_val_score(model, feature_columns, output_columns, cv=kfold)
  accuracy['RandomForest'][num_trees] = results.mean()

max_accuracy = accuracy
for key, item in accuracy.items():
  max_accuracy[key] = np.max(item)
print (max_accuracy)
  
  #   print(num_trees)
  
# print (np.shape(output_columns))
# model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
# model.fit(feature_columns, output_columns)
# print ('accuracy is ',model.score(feature_columns, output_columns))
# results = model_selection.cross_val_score(model, feature_columns, output_columns, cv=kfold)
# print (results)
# print('decision tree classifer is ' , results.mean())

# num_trees = 30
# kfold = model_selection.KFold(n_splits=10, random_state=seed)
# model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
# results = model_selection.cross_val_score(model, feature_columns, output_columns, cv=kfold)
# model.fit(feature_columns, output_columns)
# print('AdaBoostClassifier is ' , results.mean())



{'Bagging': 0.8227340823970037, 'Boosting': 0.8059300873907616, 'RandomForest': 0.809238451935081}
